In [1]:
import sys, os
from os.path import join, dirname, abspath
import matplotlib.pyplot as plt
from matplotlib.pyplot import Figure, Axes
import numpy as np
import networkx as nx
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import make_axes_locatable

from string import ascii_uppercase
plt.rcParams.update({
    "text.usetex": False,
    "ytick.minor.visible":True,
    "xtick.minor.visible":True,
    'xtick.direction': "in",
    'ytick.direction': "in"
})
outdir = "out"
os.makedirs(outdir,exist_ok=True)
def out(fname): return join(outdir,fname)
def savefig(plot_name): 
    plt.savefig(out(plot_name),bbox_inches="tight",dpi=250)
    
import pandas as pd
from numpy.linalg import matrix_power, eig

def arr_to_latex(M):
    return '$$\n' + r'\begin{bmatrix}' + '\n' + (r'\\' + '\n').join('&'.join(str(x) for x in row) for row in M) + '\n' + r'\end{bmatrix}' + '\n' +'$$'

def vec_to_latex(x,round=3):
    return '$$\n' + r'\begin{bmatrix}' + '\n' + (r' \\ ').join(str(np.round(v,round)) for v in x) + '\n' + r'\end{bmatrix}' + '\n' +'$$'

In [2]:
from car import Car
from tiles import Road, Exit
from world import World

# Test: do we restore the original state of a cell after driving through

In [3]:
GRID_WIDTH = 5
GRID_HEIGHT = 6
road_tiles = []
for i in range(0,GRID_WIDTH):
    for j in range(0,GRID_HEIGHT):
        if not ((j == 2 or j == 3) and i == 2):
            road_tiles.append(Road(i,j,np.ones(9)/9))
len(road_tiles)
car = Car(road_tiles[0],desired_speed=1)
w = World(tiles=road_tiles,cars=[car])
orig_direction = np.array(road_tiles[0].p_directions, copy=True)
print(orig_direction)
car.reset()
car.move_to(road_tiles[1])
car.reset()
car.move_to(road_tiles[2])
assert np.allclose(road_tiles[0].p_directions, orig_direction)

[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.
 0.         0.         0.16666667]
changing neighbor 0 idx 4 (prev 0.16666666666666666) of Road(0,1,occupied=False) to 0
0.16666666666666666 [0.2 0.2 0.2 0.2 0.  0.  0.  0.  0.2] [0 1 2 3 8]
[0.125 0.    0.125 0.125 0.125 0.125 0.125 0.125 0.125]
changing neighbor 1 idx 5 (prev 0.125) of Road(1,1,occupied=False) to 0
0.125 [0.14285714 0.         0.14285714 0.14285714 0.14285714 0.
 0.14285714 0.14285714 0.14285714] [0 2 3 4 6 7 8]
[0.16666667 0.16666667 0.16666667 0.         0.         0.
 0.16666667 0.16666667 0.16666667]
changing neighbor 2 idx 6 (prev 0.16666666666666666) of Road(1,0,occupied=False) to 0
0.16666666666666666 [0.2 0.2 0.2 0.  0.  0.  0.  0.2 0.2] [0 1 2 7 8]
[0.25 0.25 0.25 0.   0.   0.   0.   0.   0.25]
changing neighbor 0 idx 4 (prev 0.0) of Road(0,1,occupied=False) to 0.16666666666666666
0.16666666666666666 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.
 0.         0.         0.16666667] [0 1 2 

# Test: can we catch bad moves from the car

In [4]:
GRID_WIDTH = 5
GRID_HEIGHT = 6
road_tiles = []
for i in range(0,GRID_WIDTH):
    for j in range(0,GRID_HEIGHT):
        if not ((j == 2 or j == 3) and i == 2):
            road_tiles.append(Road(i,j,np.ones(9)/9))
car = Car(road_tiles[0],desired_speed=1)
w = World(tiles=road_tiles,cars=[car])
try:
    car.move_to(road_tiles[2])
    raise ValueError("Car should not have been able to move there!")
except ValueError:
    pass  # :)

[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.
 0.         0.         0.16666667]
changing neighbor 0 idx 4 (prev 0.16666666666666666) of Road(0,1,occupied=False) to 0
0.16666666666666666 [0.2 0.2 0.2 0.2 0.  0.  0.  0.  0.2] [0 1 2 3 8]
[0.125 0.    0.125 0.125 0.125 0.125 0.125 0.125 0.125]
changing neighbor 1 idx 5 (prev 0.125) of Road(1,1,occupied=False) to 0
0.125 [0.14285714 0.         0.14285714 0.14285714 0.14285714 0.
 0.14285714 0.14285714 0.14285714] [0 2 3 4 6 7 8]
[0.16666667 0.16666667 0.16666667 0.         0.         0.
 0.16666667 0.16666667 0.16666667]
changing neighbor 2 idx 6 (prev 0.16666666666666666) of Road(1,0,occupied=False) to 0
0.16666666666666666 [0.2 0.2 0.2 0.  0.  0.  0.  0.2 0.2] [0 1 2 7 8]
